In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def obtener_url_liga(nombre_liga):
    ligas = {
        "laliga": "https://www.transfermarkt.es/laliga/startseite/wettbewerb/ES1",
        "premier league": "https://www.transfermarkt.es/premier-league/startseite/wettbewerb/GB1",
        "serie a": "https://www.transfermarkt.es/serie-a/startseite/wettbewerb/IT1",
        "bundesliga": "https://www.transfermarkt.es/bundesliga/startseite/wettbewerb/L1",
        "ligue 1": "https://www.transfermarkt.es/ligue-1/startseite/wettbewerb/FR1"
    }
    return ligas.get(nombre_liga.lower())

def obtener_urls_equipos(url_liga):
    try:
        response = requests.get(url_liga, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error al acceder a {url_liga}: {e}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    equipos_urls = ["https://www.transfermarkt.es" + equipo['href']
                    for equipo in soup.select('td.hauptlink a[href*="/startseite/verein/"]')]
    return equipos_urls

def extraer_datos_jugadores(urls_equipos):
    data_with_url = []
    for url in urls_equipos:
        try:
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error al acceder a {url}: {e}")
            continue

        soup = BeautifulSoup(response.content, "html.parser")
        table = soup.find('table', {'class': 'items'})

        if not table:
            print(f"No se encontró la tabla en {url}")
            continue

        for row in table.select('tbody tr'):
            row_data = [td.text.strip() for td in row.find_all('td')]
            if len(row_data) >= 8:
                data_with_url.append(row_data[:8] + [url])

        time.sleep(1)  # Reduce la pausa para optimizar el scraping

    return data_with_url

def main(nombre_liga):
    url_liga = obtener_url_liga(nombre_liga)
    if not url_liga:
        print(f"No se encontró la liga '{nombre_liga}'. Verifica el nombre.")
        return None

    urls_equipos = obtener_urls_equipos(url_liga)
    data = extraer_datos_jugadores(urls_equipos)

    column_names = ['Col1', 'Col2', 'Col3', 'Jugador', 'Pos', 'Fecha Nac', 'Col7', 'Valor Mercado', 'url']
    df = pd.DataFrame(data, columns=column_names)

    df['Equipo'] = df['url'].map(lambda url: url.split('/')[3].replace('-', ' ').title())
    df.drop(columns=['Col1', 'Col2', 'Col3', 'Col7', 'url'], inplace=True)

    return df

ligas_disponibles = [
    "laliga", "premier league", "serie a", "bundesliga", "ligue 1"]

dataframes = []
for liga in ligas_disponibles:
    df_liga = main(liga)
    if df_liga is not None:
        df_liga["Competicion"] = liga.title()
        dataframes.append(df_liga)

df_total = pd.concat(dataframes, ignore_index=True)

df_total["Edad"] = df_total["Fecha Nac"].str.extract(r"\((\d+)\)")
df_total["Edad"] = pd.to_numeric(df_total["Edad"], errors='coerce').fillna(0).astype(int)
# Verificar si hay jugadores duplicados en el DataFrame
df_total.duplicated(subset=["Jugador"], keep=False)


df=df_total
df = df[df["Valor Mercado"] != "-"]
def convertir_valor_mercado(valor):
    if isinstance(valor, str):  # Asegurar que el valor es una cadena de texto
        valor = valor.replace(",", ".").strip()  # Reemplazar la coma decimal y limpiar espacios

        if "mill" in valor:  # Si el valor está en millones
            valor = valor.replace("mill. €", "").strip()
            return float(valor)  # Convertir directamente

        if "mil" in valor:  # Si el valor está en miles
            valor = valor.replace("mil. €", "").replace("mil €", "").strip()
            return float(valor) / 1000  # Convertir correctamente de miles a millones

    return None  # Si el valor no es un string, devolver None

# Aplicar la conversión en la columna "Valor Mercado" del DataFrame df2
df["Valor Mercado"] = df["Valor Mercado"].apply(convertir_valor_mercado)

df

/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_1890/3006911463.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Valor Mercado"] = df["Valor Mercado"].apply(convertir_valor_mercado)


,Jugador,Pos,Fecha Nac,Valor Mercado,Equipo,Competicion,Edad
0,Thibaut Courtois,Portero,11/05/1992 (32),25.0,Real Madrid,Laliga,32
1,Andriy Lunin,Portero,11/02/1999 (26),20.0,Real Madrid,Laliga,26
2,Fran González,Portero,24/06/2005 (19),0.4,Real Madrid,Laliga,19
3,Éder Militão,Defensa central,18/01/1998 (27),40.0,Real Madrid,Laliga,27
4,Raúl Asencio,Defensa central,13/02/2003 (22),30.0,Real Madrid,Laliga,22
...,...,...,...,...,...,...,...
2574,Estéban Lepaul,Delantero centro,18/04/2000 (24),5.0,Sco Angers,Ligue 1,24
2575,Bamba Dieng,Delantero centro,23/03/2000 (25),3.5,Sco Angers,Ligue 1,25
2576,Ibrahima Niane,Delantero centro,11/03/1999 (26),1.5,Sco Angers,Ligue 1,26
2577,Sidiki Chérif,Delantero centro,15/12/2006 (18),1.0,Sco Angers,Ligue 1,18
